In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.image as mpimg
import io
import json
import numpy as np
import time
import datetime
import functools
import matplotlib.pyplot as plt
import sys
import random
from sklearn.metrics import accuracy_score, confusion_matrix
from noise_generate import get_noisy_labels
import pandas as pd

import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt

#Text Color
from termcolor import colored

#Train Test Split
from sklearn.model_selection import train_test_split

#Model Evaluation
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from mlxtend.plotting import plot_confusion_matrix

#Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
def sample_class(x_train, y_train, x_test, y_test, train_idxes, test_idxes, classes):
    # sampling instances by sampled classes
    x_train_sampled_class = list()
    y_train_sampled_class = list()
    x_test_sampled_class = list()
    y_test_sampled_class = list()
    train_sampled_class_idxes = list()
    test_sampled_class_idxes = list()

    # For training set
    for i in range(0, len(train_idxes)):
      if y_train[i] in classes:
        x_train_sampled_class.append(x_train[i])
        y_train_sampled_class.append(y_train[i])
        train_sampled_class_idxes.append(train_idxes[i])
    # For testing set
    for i in range(0, len(test_idxes)):
      if y_test[i] in classes:
        x_test_sampled_class.append(x_test[i])
        y_test_sampled_class.append(y_test[i])
        test_sampled_class_idxes.append(test_idxes[i])

    return np.array(x_train_sampled_class), np.array(y_train_sampled_class), np.array(x_test_sampled_class), np.array(y_test_sampled_class), train_sampled_class_idxes, test_sampled_class_idxes
  
def sample_dataset_diff_frac(x_train, y_train, x_test, y_test, train_frac, test_frac):
    # get idxes list of each class in training set and testing set
    class_idx_dict_train = dict()
    for idx in range(0, y_train.shape[0]):
        if y_train[idx] not in class_idx_dict_train.keys(): class_idx_dict_train[y_train[idx]] = list()
        class_idx_dict_train[y_train[idx]].append(idx)

    class_idx_dict_test = dict()
    for idx in range(0, y_test.shape[0]):
        if y_test[idx] not in class_idx_dict_test.keys(): class_idx_dict_test[y_test[idx]] = list()
        class_idx_dict_test[y_test[idx]].append(idx)


    # sampling indexes
    class_idx_dict_train_sampled = dict()
    for cls in class_idx_dict_train.keys():
        cur_list = class_idx_dict_train[cls]
        sampled_list = random.sample(cur_list, int(train_frac * len(cur_list)))
        class_idx_dict_train_sampled[cls] = sampled_list


    class_idx_dict_test_sampled = dict()
    for cls in class_idx_dict_test.keys():
        cur_list = class_idx_dict_test[cls]
        sampled_list = random.sample(cur_list, int(test_frac * len(cur_list)))
        class_idx_dict_test_sampled[cls] = sampled_list


    # sampling instances by sampled indexes
    x_train_sampled = list()
    y_train_sampled = list()
    x_test_sampled = list()
    y_test_sampled = list()
    train_sampled_idxes = list()
    test_sampled_idxes = list()

    for idxes_list in class_idx_dict_train_sampled.values():
        train_sampled_idxes.extend(idxes_list)
    for idxes_list in class_idx_dict_test_sampled.values():
        test_sampled_idxes.extend(idxes_list)

    random.shuffle(train_sampled_idxes)
    random.shuffle(test_sampled_idxes)

    for idx in train_sampled_idxes:
        x_train_sampled.append(x_train[idx])
        y_train_sampled.append(y_train[idx])
    for idx in test_sampled_idxes:
        x_test_sampled.append(x_test[idx])
        y_test_sampled.append(y_test[idx])
    
    return np.array(x_train_sampled), np.array(y_train_sampled), np.array(x_test_sampled), np.array(y_test_sampled), train_sampled_idxes, test_sampled_idxes

def sample_dataset(x_train, y_train, x_test, y_test, frac):
    # get idxes list of each class in training set and testing set
    class_idx_dict_train = dict()
    for idx in range(0, y_train.shape[0]):
        if y_train[idx] not in class_idx_dict_train.keys(): class_idx_dict_train[y_train[idx]] = list()
        class_idx_dict_train[y_train[idx]].append(idx)

    class_idx_dict_test = dict()
    for idx in range(0, y_test.shape[0]):
        if y_test[idx] not in class_idx_dict_test.keys(): class_idx_dict_test[y_test[idx]] = list()
        class_idx_dict_test[y_test[idx]].append(idx)


    # sampling indexes
    class_idx_dict_train_sampled = dict()
    for cls in class_idx_dict_train.keys():
        cur_list = class_idx_dict_train[cls]
        sampled_list = random.sample(cur_list, int(frac * len(cur_list)))
        class_idx_dict_train_sampled[cls] = sampled_list


    class_idx_dict_test_sampled = dict()
    for cls in class_idx_dict_test.keys():
        cur_list = class_idx_dict_test[cls]
        sampled_list = random.sample(cur_list, int(frac * len(cur_list)))
        class_idx_dict_test_sampled[cls] = sampled_list


    # sampling instances by sampled indexes
    x_train_sampled = list()
    y_train_sampled = list()
    x_test_sampled = list()
    y_test_sampled = list()
    train_sampled_idxes = list()
    test_sampled_idxes = list()

    for idxes_list in class_idx_dict_train_sampled.values():
        train_sampled_idxes.extend(idxes_list)
    for idxes_list in class_idx_dict_test_sampled.values():
        test_sampled_idxes.extend(idxes_list)

    random.shuffle(train_sampled_idxes)
    random.shuffle(test_sampled_idxes)

    for idx in train_sampled_idxes:
        x_train_sampled.append(x_train[idx])
        y_train_sampled.append(y_train[idx])
    for idx in test_sampled_idxes:
        x_test_sampled.append(x_test[idx])
        y_test_sampled.append(y_test[idx])
    
    return np.array(x_train_sampled), np.array(y_train_sampled), np.array(x_test_sampled), np.array(y_test_sampled), train_sampled_idxes, test_sampled_idxes




In [ ]:
dataset = 'agnews_10pct'

In [ ]:
if not os.path.isdir('agnews_10pct_sampled_data'):
    os.mkdir('agnews_10pct_sampled_data')
if not os.path.isdir('agnews_10pct_logits_and_preds'):
    os.mkdir('agnews_10pct_logits_and_preds')
if not os.path.isdir('agnews_10pct_noisy_data'):
    os.mkdir('agnews_10pct_noisy_data')

In [ ]:
#File Path
TRAIN_FILE_PATH = "agnews_10pct_data/train_10pct_new.csv"
TEST_FILE_PATH = "agnews_10pct_data/test_10pct_new.csv"
#Load Data
traindata = pd.read_csv(TRAIN_FILE_PATH, header=None)
testdata = pd.read_csv(TEST_FILE_PATH, header=None)
#Set Column Names 
traindata.columns = ['ClassIndex', 'Title', 'Description']
testdata.columns = ['ClassIndex', 'Title', 'Description']#Combine Title and Description
x_train = traindata['Title'] + " " + traindata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_train = traindata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0
x_test = testdata['Title'] + " " + testdata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_test = testdata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0




In [ ]:
noise_ratio_list = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.45, 0.5, 0.55, 0.6, 0.65]
noise_type_list = ['uniform', 'class-dependent', 'locally-concentrated']  # corresponding to three noise types: NCAR, NAR, NNAR

**4 classes**

In [ ]:

classes = [0,1,2,3]
classStr = ''  # if the dataset contains all classes instance, then no need for specifying classes in the dataset

# datasize_list = ['full', 'frac5', 'frac2', 'frac1']
# class_frac_list = [1, 0.5, 0.2, 0.1]
datasize_list = ['frac9']
class_frac_list = [0.9]

# datasize_list = ['frac9', 'frac8', 'frac7', 'frac6', 'frac4', 'frac3']
# class_frac_list = [0.9,0.8,0.7,0.6,0.4,0.3]



num_classes = len(classes)  # has to be 3 in this loop
for j in range(0, len(datasize_list)):
  class_frac = class_frac_list[j]
  datasize = datasize_list[j]
  print("datasetsize: ", datasize)
  # datasize = 'full'  # 'frac5', 'frac2', 'frac1'
  # class_frac = 1  # 0.5, 0.2, 0.1
  # num_classes = len(classes)  # has to be 4 in this loop

  x_train_full, y_train_full, x_test_full, y_test_full, train_full_idx, test_full_idx = sample_dataset(x_train, y_train, x_test, y_test, class_frac)
  np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_train_'+ datasize +'_idx', train_full_idx)
  np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_test_'+ datasize +'_idx', test_full_idx)


  #Max Length of sentences in Train Dataset
  maxLen = max([len(item.split()) for item in np.array(x_train_full)])
  print("maxLen for the subset is: ", maxLen)


  # Tokenize and Pad data
  vocab_size = 10000 # arbitrarily chosen
  embed_size = 32 # arbitrarily chosen

  # Create and Fit tokenizer
  tok = Tokenizer(num_words=vocab_size)
  tok.fit_on_texts(x_train_full)

  # Tokenize data
  x_train_full = tok.texts_to_sequences(x_train_full)
  x_test_full = tok.texts_to_sequences(x_test_full)

  # Pad data
  x_train_full = pad_sequences(x_train_full, maxlen=maxLen)
  x_test_full = pad_sequences(x_test_full, maxlen=maxLen)


  model_full = keras.Sequential(
      [
      Embedding(vocab_size, embed_size, input_length=maxLen),
      Bidirectional(LSTM(128, return_sequences=True)),
      Bidirectional(LSTM(64, return_sequences=True)),
      GlobalMaxPooling1D(),
      #  Dense(1024),
      #  Dropout(0.25),
      #  Dense(512),
      #  Dropout(0.25),
      Dense(256),
      Dropout(0.25),
      Dense(128),
      Dropout(0.25),
      Dense(64),
      Dropout(0.25),
      Dense(4),
      Activation('softmax')
      ]
  )
  model_full.summary()



  callbacks = [
      EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
          monitor='val_accuracy',
          min_delta=1e-4,
          patience=6,
          verbose=1
      ),
      ModelCheckpoint(
          filepath='weights.h5',
          monitor='val_accuracy', 
          mode='max', 
          save_best_only=True,
          save_weights_only=True,
          verbose=1
      )
  ]

  batch_size=256  #256
  epochs=20   #20
  model_full.compile(loss='sparse_categorical_crossentropy', #Sparse Categorical Crossentropy Loss because data is not one-hot encoded
                optimizer='adam', 
                metrics=['accuracy']) 

  model_full.fit(x_train_full, 
            y_train_full, 
            batch_size=batch_size, 
            validation_data=(x_test_full, y_test_full), 
            epochs=epochs, 
            callbacks=callbacks)



  predict_res_full = model_full.predict(x_test_full)
  y_pred_full = np.argmax(predict_res_full,axis=1)
  accuracy_full = accuracy_score(y_test_full, y_pred_full)

  model_penultimate_full = tf.keras.Model(model_full.layers[0].input, model_full.layers[-2].output)  # model soft
  model_last_full = model_full.layers[-1]
  logits_train_full = model_penultimate_full(x_train_full)
  soft_pred_train_full = model_last_full(logits_train_full)
  np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_logits_train', logits_train_full)
  np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_train', soft_pred_train_full)
  np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_test', predict_res_full)


  for noise_type in noise_type_list:
    for noise_ratio in noise_ratio_list:
      y_train_noisy, probs = get_noisy_labels(dataset, x_train_full, y_train_full, x_test_full, y_test_full, num_classes, datasize, noise_type, noise_ratio, classStr)
      np.save(dataset + '_noisy_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_' + datasize + '_' + noise_type + '_' + str(int(noise_ratio*100)), y_train_noisy)


**3 Classes**

In [ ]:
#File Path
TRAIN_FILE_PATH = "agnews_10pct_data/train_10pct_new.csv"
TEST_FILE_PATH = "agnews_10pct_data/test_10pct_new.csv"
#Load Data
traindata = pd.read_csv(TRAIN_FILE_PATH, header=None)
testdata = pd.read_csv(TEST_FILE_PATH, header=None)
#Set Column Names 
traindata.columns = ['ClassIndex', 'Title', 'Description']
testdata.columns = ['ClassIndex', 'Title', 'Description']#Combine Title and Description
x_train = traindata['Title'] + " " + traindata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_train = traindata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0
x_test = testdata['Title'] + " " + testdata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_test = testdata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0




In [ ]:

classes_list = [[0,1,2], [1,2,3], [0,1,3], [0,2,3]]
classStr_list = ['012', '123', '013', '023']



for i in range(0, len(classes_list)):
  classes = classes_list[i]
  classStr = classStr_list[i]


  # print("CURRENT CLASSSTR ===================================: ", classStr)
  # datasize_list = ['frac5', 'frac2', 'frac1']
  # class_frac_list = [10/15, 4/15, 2/15]

  datasize_list = ['frac7', 'frac6', 'frac4', 'frac3']
  class_frac_list = [14/15, 12/15, 8/15, 6/15]

  num_classes = len(classes)  # has to be 3 in this loop
  for j in range(0, len(datasize_list)):
    class_frac = class_frac_list[j]
    datasize = datasize_list[j]

    x_train_classFrac, y_train_classFrac, x_test_classFrac, y_test_classFrac, train_classFrac_idx, test_classFrac_idx = sample_dataset(x_train, y_train, x_test, y_test, class_frac)
    x_train_classFrac_3cls, y_train_classFrac_3cls, x_test_classFrac_3cls, y_test_classFrac_3cls, train_classFrac_3cls_idx, test_classFrac_3cls_idx = sample_class(x_train_classFrac, y_train_classFrac, x_test_classFrac, y_test_classFrac, train_classFrac_idx, test_classFrac_idx, classes)
    
    # print("CURRENT class_Frac ===================================: ", class_frac)
    # print("CURRENT dataset size ===================================: ", datasize)
    # print("CURRENT class_distribution ===================================: ", np.unique(y_train_classFrac_3cls, return_counts=True))

    
    
    np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_train_'+ datasize +'_idx', train_classFrac_3cls_idx)
    np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_test_'+ datasize +'_idx', test_classFrac_3cls_idx)

      # encode the labels
    label_map = dict()
    for i in range(0, len(classes)):
      label_map[classes[i]] = i
    for i in range(0, y_train_classFrac_3cls.shape[0]):
      prev_label = y_train_classFrac_3cls[i]
      y_train_classFrac_3cls[i] = label_map[prev_label]
    for i in range(0, y_test_classFrac_3cls.shape[0]):
      prev_label = y_test_classFrac_3cls[i]
      y_test_classFrac_3cls[i] = label_map[prev_label]



    #Max Length of sentences in Train Dataset
    maxLen = max([len(item.split()) for item in np.array(x_train_classFrac_3cls)])
    # print("maxLen for the subset is: ", maxLen)


    # Tokenize and Pad data
    vocab_size = 10000 # arbitrarily chosen
    embed_size = 32 # arbitrarily chosen

    # Create and Fit tokenizer
    tok = Tokenizer(num_words=vocab_size)
    tok.fit_on_texts(x_train_classFrac_3cls)

    # Tokenize data
    x_train_classFrac_3cls = tok.texts_to_sequences(x_train_classFrac_3cls)
    x_test_classFrac_3cls = tok.texts_to_sequences(x_test_classFrac_3cls)

    # Pad data
    x_train_classFrac_3cls = pad_sequences(x_train_classFrac_3cls, maxlen=maxLen)
    x_test_classFrac_3cls = pad_sequences(x_test_classFrac_3cls, maxlen=maxLen)


    model_classFrac_3cls = keras.Sequential(
        [
        Embedding(vocab_size, embed_size, input_length=maxLen),
        Bidirectional(LSTM(128, return_sequences=True)),
        Bidirectional(LSTM(64, return_sequences=True)),
        GlobalMaxPooling1D(),
        #  Dense(1024),
        #  Dropout(0.25),
        #  Dense(512),
        #  Dropout(0.25),
        Dense(256),
        Dropout(0.25),
        Dense(128),
        Dropout(0.25),
        Dense(64),
        Dropout(0.25),
        Dense(4),
        Activation('softmax')
        ]
    )
    # model_classFrac_3cls.summary()



    callbacks = [
        EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
            monitor='val_accuracy',
            min_delta=1e-4,
            patience=6,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='weights.h5',
            monitor='val_accuracy', 
            mode='max', 
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]

    batch_size=256  #256
    epochs=20   #20
    model_classFrac_3cls.compile(loss='sparse_categorical_crossentropy', #Sparse Categorical Crossentropy Loss because data is not one-hot encoded
                  optimizer='adam', 
                  metrics=['accuracy']) 

    model_classFrac_3cls.fit(x_train_classFrac_3cls, 
              y_train_classFrac_3cls, 
              batch_size=batch_size, 
              validation_data=(x_test_classFrac_3cls, y_test_classFrac_3cls), 
              epochs=epochs, 
              callbacks=callbacks)



    predict_res_classFrac_3cls = model_classFrac_3cls.predict(x_test_classFrac_3cls)
    y_pred_classFrac_3cls = np.argmax(predict_res_classFrac_3cls,axis=1)
    accuracy_classFrac_3cls = accuracy_score(y_test_classFrac_3cls, y_pred_classFrac_3cls)

    model_penultimate_classFrac_3cls = tf.keras.Model(model_classFrac_3cls.layers[0].input, model_classFrac_3cls.layers[-2].output)  # model soft
    model_last_classFrac_3cls = model_classFrac_3cls.layers[-1]
    logits_train_classFrac_3cls = model_penultimate_classFrac_3cls(x_train_classFrac_3cls)
    soft_pred_train_classFrac_3cls = model_last_classFrac_3cls(logits_train_classFrac_3cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_logits_train', logits_train_classFrac_3cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_train', soft_pred_train_classFrac_3cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_test', predict_res_classFrac_3cls)

    for noise_type in noise_type_list:
      for noise_ratio in noise_ratio_list:
        y_train_noisy, probs = get_noisy_labels(dataset, x_train_classFrac_3cls, y_train_classFrac_3cls, x_test_classFrac_3cls, y_test_classFrac_3cls, num_classes, datasize, noise_type, noise_ratio, classStr)
        np.save(dataset + '_noisy_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_' + datasize + '_' + noise_type + '_' + str(int(noise_ratio*100)), y_train_noisy)


**2 classes**

In [ ]:
#File Path
TRAIN_FILE_PATH = "agnews_10pct_data/train_10pct_new.csv"
TEST_FILE_PATH = "agnews_10pct_data/test_10pct_new.csv"
#Load Data
traindata = pd.read_csv(TRAIN_FILE_PATH, header=None)
testdata = pd.read_csv(TEST_FILE_PATH, header=None)
#Set Column Names 
traindata.columns = ['ClassIndex', 'Title', 'Description']
testdata.columns = ['ClassIndex', 'Title', 'Description']#Combine Title and Description
x_train = traindata['Title'] + " " + traindata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_train = traindata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0
x_test = testdata['Title'] + " " + testdata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_test = testdata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0




In [ ]:

classes_list = [[0,1], [0,2], [0,3], [1,2], [1,3], [2,3]]
classStr_list = ['01', '02', '03', '12', '13', '23']



for i in range(0, len(classes_list)):
  classes = classes_list[i]
  classStr = classStr_list[i]


  # print("CURRENT CLASSSTR ===================================: ", classStr)
  # datasize_list = ['frac5', 'frac2', 'frac1']
  # class_frac_list = [1, 2/5, 1/5]
  datasize_list = ['frac4', 'frac3']
  class_frac_list = [4/5, 3/5]

  num_classes = len(classes)  # has to be 3 in this loop
  for j in range(0, len(datasize_list)):
    class_frac = class_frac_list[j]
    datasize = datasize_list[j]

    x_train_classFrac, y_train_classFrac, x_test_classFrac, y_test_classFrac, train_classFrac_idx, test_classFrac_idx = sample_dataset(x_train, y_train, x_test, y_test, class_frac)
    x_train_classFrac_2cls, y_train_classFrac_2cls, x_test_classFrac_2cls, y_test_classFrac_2cls, train_classFrac_2cls_idx, test_classFrac_2cls_idx = sample_class(x_train_classFrac, y_train_classFrac, x_test_classFrac, y_test_classFrac, train_classFrac_idx, test_classFrac_idx, classes)
    
    # print("CURRENT class_Frac ===================================: ", class_frac)
    # print("CURRENT dataset size ===================================: ", datasize)
    # print("CURRENT class_distribution ===================================: ", np.unique(y_train_classFrac_2cls, return_counts=True))

    
    
    np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_train_'+ datasize +'_idx', train_classFrac_2cls_idx)
    np.save(dataset + '_sampled_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_test_'+ datasize +'_idx', test_classFrac_2cls_idx)

      # encode the labels
    label_map = dict()
    for i in range(0, len(classes)):
      label_map[classes[i]] = i
    for i in range(0, y_train_classFrac_2cls.shape[0]):
      prev_label = y_train_classFrac_2cls[i]
      y_train_classFrac_2cls[i] = label_map[prev_label]
    for i in range(0, y_test_classFrac_2cls.shape[0]):
      prev_label = y_test_classFrac_2cls[i]
      y_test_classFrac_2cls[i] = label_map[prev_label]



    #Max Length of sentences in Train Dataset
    maxLen = max([len(item.split()) for item in np.array(x_train_classFrac_2cls)])
    # print("maxLen for the subset is: ", maxLen)


    # Tokenize and Pad data
    vocab_size = 10000 # arbitrarily chosen
    embed_size = 32 # arbitrarily chosen

    # Create and Fit tokenizer
    tok = Tokenizer(num_words=vocab_size)
    tok.fit_on_texts(x_train_classFrac_2cls)

    # Tokenize data
    x_train_classFrac_2cls = tok.texts_to_sequences(x_train_classFrac_2cls)
    x_test_classFrac_2cls = tok.texts_to_sequences(x_test_classFrac_2cls)

    # Pad data
    x_train_classFrac_2cls = pad_sequences(x_train_classFrac_2cls, maxlen=maxLen)
    x_test_classFrac_2cls = pad_sequences(x_test_classFrac_2cls, maxlen=maxLen)


    model_classFrac_2cls = keras.Sequential(
        [
        Embedding(vocab_size, embed_size, input_length=maxLen),
        Bidirectional(LSTM(128, return_sequences=True)),
        Bidirectional(LSTM(64, return_sequences=True)),
        GlobalMaxPooling1D(),
        #  Dense(1024),
        #  Dropout(0.25),
        #  Dense(512),
        #  Dropout(0.25),
        Dense(256),
        Dropout(0.25),
        Dense(128),
        Dropout(0.25),
        Dense(64),
        Dropout(0.25),
        Dense(4),
        Activation('softmax')
        ]
    )
    # model_classFrac_2cls.summary()



    callbacks = [
        EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
            monitor='val_accuracy',
            min_delta=1e-4,
            patience=6,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='weights.h5',
            monitor='val_accuracy', 
            mode='max', 
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]

    batch_size=256  #256
    epochs=20   #20
    model_classFrac_2cls.compile(loss='sparse_categorical_crossentropy', #Sparse Categorical Crossentropy Loss because data is not one-hot encoded
                  optimizer='adam', 
                  metrics=['accuracy']) 

    model_classFrac_2cls.fit(x_train_classFrac_2cls, 
              y_train_classFrac_2cls, 
              batch_size=batch_size, 
              validation_data=(x_test_classFrac_2cls, y_test_classFrac_2cls), 
              epochs=epochs, 
              callbacks=callbacks)



    predict_res_classFrac_2cls = model_classFrac_2cls.predict(x_test_classFrac_2cls)
    y_pred_classFrac_2cls = np.argmax(predict_res_classFrac_2cls,axis=1)
    accuracy_classFrac_2cls = accuracy_score(y_test_classFrac_2cls, y_pred_classFrac_2cls)

    model_penultimate_classFrac_2cls = tf.keras.Model(model_classFrac_2cls.layers[0].input, model_classFrac_2cls.layers[-2].output)  # model soft
    model_last_classFrac_2cls = model_classFrac_2cls.layers[-1]
    logits_train_classFrac_2cls = model_penultimate_classFrac_2cls(x_train_classFrac_2cls)
    soft_pred_train_classFrac_2cls = model_last_classFrac_2cls(logits_train_classFrac_2cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_logits_train', logits_train_classFrac_2cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_train', soft_pred_train_classFrac_2cls)
    np.save(dataset + '_logits_and_preds/' + dataset + '_'+ datasize +'_'+ str(num_classes) + 'cls' + classStr + '_soft_pred_test', predict_res_classFrac_2cls)

    for noise_type in noise_type_list:
      for noise_ratio in noise_ratio_list:
        y_train_noisy, probs = get_noisy_labels(dataset, x_train_classFrac_2cls, y_train_classFrac_2cls, x_test_classFrac_2cls, y_test_classFrac_2cls, num_classes, datasize, noise_type, noise_ratio, classStr)
        np.save(dataset + '_noisy_data/' + dataset + '_'+ str(num_classes) + 'cls' + classStr + '_' + datasize + '_' + noise_type + '_' + str(int(noise_ratio*100)), y_train_noisy)
